Installing the Dependencies

In [2]:
%pip install -U bitsandbytes transformers accelerate

Note: you may need to restart the kernel to use updated packages.


Logging in to Hugging Face (one time only)

In [2]:
from huggingface_hub import login
login()

Loading in the LLM (Gemma First)

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
LLM = "google/gemma-3-4b-it" # we can simply chamge this to the mistral model later

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(LLM)
model = AutoModelForCausalLM.from_pretrained(LLM,device_map="auto", quantization_config=config)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

prompt = "what color is a banana" # the prompt we are feeding in to the LLM
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
output = model.generate(**inputs,max_new_tokens=100,do_sample=False)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

what color is a banana?

This is a trick question! Bananas are yellow when ripe.

Do you want to try another riddle?


In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
LLM = "mistralai/Mistral-7B-Instruct-v0.2" # we can simply chamge this to the mistral model later

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
 
tokenizer = AutoTokenizer.from_pretrained(LLM)
model = AutoModelForCausalLM.from_pretrained(LLM,device_map="auto", quantization_config=config)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

prompt = "what color is a banana" # the prompt we are feeding in to the LLM
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
output = model.generate(**inputs,max_new_tokens=100,do_sample=False)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


what color is a banana?

A banana is not a color, it is a fruit. Bananas come in various shades of yellow when ripe, with green being the unripe stage. The color of a banana depends on its ripeness. When it is unripe, it is green, and as it ripens, it turns yellow. The inside of a ripe banana is typically creamy white.


In [17]:
from pathlib import Path
import json
import torch
import time
import re
from data_serialization import serialize_schema, build_text2sql_prompt


# Paths to Spider data
PROJECT_ROOT = Path.cwd().parents[1]
SPIDER_DIR = PROJECT_ROOT / "spider" / "spider_data"
DEV_JSON = SPIDER_DIR / "dev.json"
TABLES_JSON = SPIDER_DIR / "tables.json"
DB_DIR = SPIDER_DIR / "database"


# Load Spider JSON files
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


dev_examples = load_json(DEV_JSON)      
tables = load_json(TABLES_JSON)       
db_index = {item["db_id"]: item for item in tables}


print("Loaded dev examples:", len(dev_examples))
print("Loaded schemas:", len(tables))


# Clean SQL output
def clean_sql(text: str) -> str:
    t = text.strip()
    t = re.sub(r"^```(?:sql)?\s*", "", t, flags=re.IGNORECASE)
    t = re.sub(r"\s*```$", "", t)
    t = re.sub(r"^\s*SQL\s*:\s*", "", t, flags=re.IGNORECASE)

    m = re.search(r"\b(WITH|SELECT)\b", t, flags=re.IGNORECASE)
    if m:
        t = t[m.start():]
    return t.strip()


# Output file setup
OUT_DIR = PROJECT_ROOT / "runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)


MODEL_NAME = getattr(model.config, "_name_or_path", "model").split("/")[-1]
N = 100
OUT_PATH = OUT_DIR / f"{MODEL_NAME}_dev_first{N}.jsonl"


# Run inference
model.eval()
device = next(model.parameters()).device


print("Running model:", MODEL_NAME)
print("Device:", device)
print("Saving to:", OUT_PATH)


with open(OUT_PATH, "w", encoding="utf-8") as f:
    for i, example in enumerate(dev_examples[:N]):
        db_id = example["db_id"]
        question = example["question"]
        gold_sql = example["query"]
        schema_text = serialize_schema(db_index[db_id])
        prompt = build_text2sql_prompt(question, schema_text)
        messages = [{"role": "user", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        t0 = time.time()
        with torch.inference_mode():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=150,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        latency = time.time() - t0

        prompt_len = inputs["input_ids"].shape[-1]
        gen_only = generated_ids[0, prompt_len:]
        pred_sql_raw = tokenizer.decode(gen_only, skip_special_tokens=True)
        pred_sql = clean_sql(pred_sql_raw)

        record = {
            "i": i,
            "db_id": db_id,
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "latency_s": latency,
            "model": MODEL_NAME
        }
        f.write(json.dumps(record) + "\n")

        if i < 3:
            print("=== QUESTION ===")
            print(question)
            print("\n=== GOLD SQL ===")
            print(gold_sql)
            print("\n=== PRED SQL ===")
            print(pred_sql)
            print("\n" + "=" * 80 + "\n")

        #Progress Since Bored
        if (i + 1) % 25 == 0:
            print(f"{MODEL_NAME}: finished {i+1}/{N}")


print("Finished. Wrote file:", OUT_PATH)



Loaded dev examples: 1034
Loaded schemas: 166
Running model: gemma-3-4b-it
Device: cuda:0
Saving to: c:\Users\jesse\Desktop\CS175\CS-175-Project\runs\gemma-3-4b-it_dev_first100.jsonl
=== QUESTION ===
How many singers do we have?

=== GOLD SQL ===
SELECT count(*) FROM singer

=== PRED SQL ===
SELECT count(Singer_ID) FROM singer


=== QUESTION ===
What is the total number of singers?

=== GOLD SQL ===
SELECT count(*) FROM singer

=== PRED SQL ===
SELECT count(Singer_ID) FROM singer


=== QUESTION ===
Show name, country, age for all singers ordered by age from the oldest to the youngest.

=== GOLD SQL ===
SELECT name ,  country ,  age FROM singer ORDER BY age DESC

=== PRED SQL ===
SELECT Name, Country, Age FROM singer ORDER BY Age DESC


gemma-3-4b-it: finished 25/100
gemma-3-4b-it: finished 50/100
gemma-3-4b-it: finished 75/100
gemma-3-4b-it: finished 100/100
Finished. Wrote file: c:\Users\jesse\Desktop\CS175\CS-175-Project\runs\gemma-3-4b-it_dev_first100.jsonl


So the issue between the bottom versus the top one is the prompt as plain using tokenizer(prompt,) but Gemma-IT expect chat formatted input, so it didn't recognize that it shouild produce an answer and instead kept repeating the prompt. It also decoded the entire output sequence, which included both original prompt and the generated text, making it look like the model was just echoing instructions. the top one applies chat template to structure the prompt like a conversation.

In [29]:
print("Current model:", model.config._name_or_path)
print("Device:", next(model.parameters()).device)


Current model: mistralai/Mistral-7B-Instruct-v0.2
Device: cuda:0


In [30]:
import json
import sqlite3
import pandas as pd
from pathlib import Path

JSONL_PATH = OUT_PATH
rows = []
with open(JSONL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))
df = pd.DataFrame(rows)
print("Loaded rows:", len(df))


dev_df_all = pd.DataFrame(dev_examples)
diff_col = None
for c in ["hardness", "difficulty"]:
    if c in dev_df_all.columns:
        diff_col = c
        break
if diff_col:
    dev_df = dev_df_all[["question", "db_id", diff_col]].rename(
        columns={diff_col: "difficulty"}
    )
else:
    # Some Spider downloads don't include difficulty
    dev_df = dev_df_all[["question", "db_id"]].copy()
    dev_df["difficulty"] = "unknown"

df = df.merge(dev_df, on=["question", "db_id"], how="left")


# Exact Match (EM)
def normalize_sql(s):
    return " ".join(s.lower().split())
df["EM"] = (
    df["pred_sql"].apply(normalize_sql)
    ==
    df["gold_sql"].apply(normalize_sql)
)


# 4) Execution Accuracy (EX)
def run_sql(db_id, sql):
    db_path = DB_DIR / db_id / f"{db_id}.sqlite"
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(sql)
    res = cur.fetchall()
    conn.close()
    return res

def exec_match(row):
    try:
        pred_res = run_sql(row["db_id"], row["pred_sql"])
        gold_res = run_sql(row["db_id"], row["gold_sql"])
        return pred_res == gold_res
    except:
        return False

print("Running execution evaluation...")
df["EX"] = df.apply(exec_match, axis=1)


CSV_PATH = OUT_DIR / f"{MODEL_NAME}_evaluation.csv"
df.to_csv(CSV_PATH, index=False)

print("Saved evaluation CSV to:", CSV_PATH)

Loaded rows: 100
Running execution evaluation...
Saved evaluation CSV to: c:\Users\jesse\Desktop\CS175\CS-175-Project\runs\gemma-3-4b-it_evaluation.csv


In [15]:
from pathlib import Path

for p in Path.cwd().parents[4].rglob("dev.json"):
    print(p)

c:\Users\jesse\Desktop\CS175\CS-175-Project\spider\spider_data\dev.json


In [ ]:
import torch
#Debug to see if runs on 4090
print("cuda available:", torch.cuda.is_available())
print("device of first real param:", next(p for p in model.parameters() if p.device.type != "meta").device)


cuda available: True


NameError: name 'model' is not defined